### Introduction

Considering that porting the inferencing from an Azure ML Endpoints workflow to a Local Workflow was required to be done [see earlier Notebook titled LocalLlama-LangChain which tested the various models and approaches](https://github.com/sankalpsrv/Schematise/blob/main/Notebook-of-approaches/LocalLlama-LangChain.ipynb)this notebook documents the process of alignment of the code so far with a Local inference workflow on a Cloud GPU endpoint with 24 GB RAM (GPU)

### Contents
[Part-1](#Part-1) Setup using GPTQ model from the Bloke -with instructions on HuggingFace model page.

[Part-2](#Part-2) Porting the Send_request function using LangChain's FewShotChatMessagePromptTemplate class

[Part-3-A](#Part-3-A) Trying an approach for prompting using SystemMessagePromptTemplate

[Part-3-B](#Part-3-B) Prompting the ChatPromptTemplate by adding System Prompt to Llama2's instruction prompt

### Results

1. The context length for prompting using SystemMessagePromptTemplate was lesser than when using the Llama2 Instruction Prompt Template in the "Human Message". This signified that the context was missing. Hence, it the latter strategy yields better results (see last cell)

2. Setting up a pipeline with parameters may have to be reconsidered in light of the warning message and deprecation.

3. While the send_request function was successfully ported, further strategies may involve using ExampleSelector on LangChain




### Part-1

Setup using GPTQ model from the Bloke -with instructions on HuggingFace model page.

In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
def instantiate_model():
    model_name_or_path = "TheBloke/Llama-2-7B-Chat-GPTQ"
    # To use a different branch, change revision
    # For example: revision="main"
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                                 device_map="auto",
                                                 trust_remote_code=True,
                                                 revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=4000,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1
    )
    hf = HuggingFacePipeline(pipeline=pipe)

    return hf

/root/anaconda3/envs/Schematise-dev-server/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Part-2 

Porting the Send_request function using LangChain's FewShotChatMessagePromptTemplate class

In [12]:
    def send_request(hf, section_for_conversion):
        examples = [
            {
                "question": "Convert this statute extract to Legal Rule ML XML format: The benefit referred to in comma 1 shall be paid in an amount equal 80 per cent of five-twelfths of the income earned and reported for tax purposes by the freelancer in the second year preceding the year of application.",
                "answer": """<lrml:ConstitutiveStatement key="tax2">

          <ruleml:Rule closure="universal">

        <lrml:Paraphrase>If income is reported in some year, then it was earned in the previous year.

        </lrml:Paraphrase>

        <ruleml:if>

          <ruleml:Atom>

            <ruleml:Rel iri=":reported"/>

            <ruleml:Var>income</ruleml:Var>

            <ruleml:Var>year</ruleml:Var>

          </ruleml:Atom>

        </ruleml:if>

        <ruleml:then>

          <ruleml:Atom>

            <ruleml:Rel iri=":earned"/>

            <ruleml:Var>income</ruleml:Var>

            <ruleml:Expr key=":year-1">

              <ruleml:Fun iri=":subtract"/>

              <ruleml:Var>year</ruleml:Var>

              <ruleml:Data xsi:type="xs:integer">1</ruleml:Data>

            </ruleml:Expr>

          </ruleml:Atom>

        </ruleml:then>

      </ruleml:Rule>

    </lrml:ConstitutiveStatement>
    <lrml:PrescriptiveStatement key="literal">

    <ruleml:Rule closure="universal" key=":literal-template">

        <ruleml:if>

            <ruleml:And>

                <ruleml:Atom key=":atom-earned">

                    <ruleml:Rel iri=":earned"/>

                     <ruleml:Var>income</ruleml:Var>

                     <ruleml:Expr>

                        <ruleml:Fun iri=":subtract"/>

                        <ruleml:Var>year</ruleml:Var>

                        <ruleml:Data xsi:type="xs:integer">2</ruleml:Data>

                     </ruleml:Expr>

                 </ruleml:Atom>

                 <ruleml:Atom key=":atom-reported">

                     <ruleml:Rel iri=":reported"/>

                     <ruleml:Var>income</ruleml:Var>

                    <ruleml:Expr>

                         <ruleml:Fun iri=":subtract"/>

                         <ruleml:Var>year</ruleml:Var>

                         <ruleml:Data xsi:type="xs:integer">2</ruleml:Data>

                     </ruleml:Expr>

                 </ruleml:Atom>

            </ruleml:And>

        </ruleml:if>

        <ruleml:then>

             <lrml:Obligation key="obl-paybenefit">

                  <ruleml:slot>

                      <lrml:Bearer/>

                      <ruleml:Var>Employer</ruleml:Var>

                  </ruleml:slot>

                  <ruleml:slot>

                      <lrml:AuxiliaryParty/>

                      <ruleml:Var>Freelancer</ruleml:Var>

                  </ruleml:slot>

                  <ruleml:Atom>

                      <ruleml:Rel iri=":paybenefit"/>

                     <ruleml:Expr>

                          <ruleml:Fun iri=":80_percent_of_five-twelfths_of"/>

                          <ruleml:Var>income</ruleml:Var>

                      </ruleml:Expr>

                      <ruleml:Var>year</ruleml:Var>

                  </ruleml:Atom>

              </lrml:Obligation>

          </ruleml:then>

      </ruleml:Rule>

</lrml:PrescriptiveStatement>"""
            },
            {
                "question": "Convert this statute extract to Legal Rule ML XML format:A person must not engage in a credit activity if the person does not hold a licence authorising the person to engage in the credit activity. Civil penalty: 2,000 penalty units.  Criminal penalty: 200 penalty units, or 2 years imprisonment, or both.",
                "answer": """
                <lrml:LegalSources>

   <lrml:LegalSource key="ls1"             sameAs="http://www.comlaw.gov.au/Details/C2009A00134/Html/Text#param43"/>

</lrml:LegalSources>


<lrml:Context key="psInfo1">

   <lrml:appliesAssociations>

          <lrml:Associations>

                <lrml:Association>

                       <lrml:appliesSource keyref="#ls1/>

                       <lrml:toTarget keyref="#ps1"/>

                       <lrml:toTarget keyref="#ps2"/>

                       <lrml:toTarget keyref="#pen1"/>

                       <lrml:toTarget keyref="#pen2"/>

                </lrml:Association>

          </lrml:Associations>

</lrml:appliesAssociations>

</lrml:Context>


<lrml:Statements key="textblock1">

<lrml:hasQualification>

  <lrml:Override over="#ps2" under="#ps1"/>

  </lrml:hasQualification>

  <lrml:PrescriptiveStatement key="ps1">

      <ruleml:Rule key=":rule1" closure="universal">

        <lrml:hasStrength>

          <lrml:DefeasibleStrength key="str1" iri="&defeasible-ontology;#defeasible1"/>

        </lrml:hasStrength>

        <ruleml:if>

          <ruleml:Atom>

            <ruleml:Rel iri=":person"/>

            <ruleml:Var>X</ruleml:Var>

          </ruleml:Atom>

        </ruleml:if>

        <ruleml:then>

          <lrml:SuborderList>

            <lrml:Prohibition>

              <ruleml:Atom>

                <ruleml:Rel iri=":engageCreditActivity"/>

                <ruleml:Var>X</ruleml:Var>

              </ruleml:Atom>

            </lrml:Prohibition>

          </lrml:SuborderList>

        </ruleml:then>

      </ruleml:Rule>

    </lrml:PrescriptiveStatement>



    <lrml:PenaltyStatement key="pen2">

      <lrml:SuborderList>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":payPenalUnits"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>200</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":imprisonment"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>2 years</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":payPenalUnitPlusImprisonment"/>

            <ruleml:Var>X</ruleml:Var>

            <ruleml:Ind>200</ruleml:Ind>

            <ruleml:Ind>2 years</ruleml:Ind>

          </ruleml:Atom>

        </lrml:Obligation>

      </lrml:SuborderList>

    </lrml:PenaltyStatement>



    <lrml:ReparationStatement key="rep1">

      <lrml:Reparation key="assoc1">

        <lrml:appliesPenalty keyref="#pen1"/>

        <lrml:toPrescriptiveStatement keyref="#ps1"/>

      </lrml:Reparation>

    </lrml:ReparationStatement>

    <lrml:ReparationStatement key="rep2">

      <lrml:Reparation keyref="#assoc1">

        <lrml:appliesPenalty keyref="#pen2"/>

        <lrml:toPrescriptiveStatement keyref="#ps1"/>

      </lrml:Reparation>

    </lrml:ReparationStatement>

</lrml:Statements>
                """
            }
        ]
        example_prompt = ChatPromptTemplate.from_messages(
            [
                ("human", "{question}"),
                ("ai", "{answer}"),
            ]

        )

        few_shot_prompt = FewShotChatMessagePromptTemplate(
            example_prompt=example_prompt,
            examples=examples,
        )

        final_prompt = ChatPromptTemplate.from_messages(
            [
                ("system",
                 "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same with reference to the documentation available at http://docs.oasis-open.org/legalruleml/legalruleml-core-spec/v1.0/legalruleml-core-spec-v1.0.html"),
                few_shot_prompt,
                ("human", "{question}"),
            ]
        )

        llm_chain = final_prompt | hf

        result = llm_chain.invoke({"question": section_for_conversion})

        return result

In [2]:
hf = instantiate_model()

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.chains import LLMChain

In [15]:
section_for_conversion = "6. Responsibilities of collection centres.	collect e-waste on behalf of producer or dismantler or recycler or refurbisher including those arising from orphaned products;, Provided the collection centres established by producer can also collect e-waste on behalf of dismantler, refurbisher and recycler including those arising from orphaned products, ensure that the facilities are in accordance with the standards or guidelines issued by Central Pollution Control Board from time to time;, ensure that the e-waste collected by them is stored in a secured manner till it is sent to authorised dismantler or recycler as the case may be;, ensure that no damage is caused to the environment during storage and transportation of e-waste;, maintain records in Form-2 of the e-waste handled as per the guidelines of Central Pollution Control Board and make such records available for scrutiny by the Central Pollution Control Board or the concerned State Pollution Control Board as and when asked for."

result = send_request(hf, section_for_conversion)

In [17]:
print(result)



AI: <lrml:ConstitutiveStatement key="statuteExtract">

<ruleml:Rule closure="universal">

    <lrml:Paraphrase>The collection centers must collect e-waste on behalf of producers, dismantlers, recyclers, and refurbishers, including waste from orphaned products.</lrml:Paraphrase>

    <ruleml:if>

        <ruleml:Atom>

            <ruleml:Rel iri=":collectionCenter"/>

            <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

    </ruleml:if>

    <ruleml:then>

      <lrml:Obligation>

        <ruleml:Atom>

          <ruleml:Rel iri=":storeEWasteSecurely"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

        <ruleml:Atom>

          <ruleml:Rel iri=":ensureNoEnvironmentalDamage"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

        <ruleml:Atom>

          <ruleml:Rel iri=":maintainRecordsForm2"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

      </lrml:Obligation>

    </ruleml:then>

</ruleml:Rule>

</lrml:Constitutive

In [16]:
print(result)


AI: <lrml:ConstitutiveStatement key="collectioncentre">

      <ruleml:Rule closure="universal">

    <lrml:Paraphrase>Collection centres must collect e-waste on behalf of producers, dismantlers, refurbishers, and recyclers, including waste from orphaned products.</lrml:Paraphrase>

    <ruleml:if>

      <ruleml:Atom>

        <ruleml:Rel iri=":establishFacility"/>

        <ruleml:Var>X</ruleml:Var>

      </ruleml:Atom>

    </ruleml:if>

    <ruleml:then>

      <lrml:SuborderList>

        <lrml:Prohibition>

          <ruleml:Atom>

            <ruleml:Rel iri=":storeEWasteSecurely"/>

            <ruleml:Var>X</ruleml:Var>

          </ruleml:Atom>

        </lrml:Prohibition>

      </lrml:SuborderList>

    </ruleml:then>

    <ruleml:then>

      <lrml:SuborderList>

        <lrml:Obligation>

          <ruleml:Atom>

            <ruleml:Rel iri=":transportEWasteSecurely"/>

            <ruleml:Var>X</ruleml:Var>

          </ruleml:Atom>

        </lrml:Obligation>

      <

In [4]:
result= """<lrml:ConstitutiveStatement key="statuteExtract">

<ruleml:Rule closure="universal">

    <lrml:Paraphrase>The collection centers must collect e-waste on behalf of producers, dismantlers, recyclers, and refurbishers, including waste from orphaned products.</lrml:Paraphrase>

    <ruleml:if>

        <ruleml:Atom>

            <ruleml:Rel iri=":collectionCenter"/>

            <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

    </ruleml:if>

    <ruleml:then>

      <lrml:Obligation>

        <ruleml:Atom>

          <ruleml:Rel iri=":storeEWasteSecurely"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

        <ruleml:Atom>

          <ruleml:Rel iri=":ensureNoEnvironmentalDamage"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

        <ruleml:Atom>

          <ruleml:Rel iri=":maintainRecordsForm2"/>

          <ruleml:Var>X</ruleml:Var>

        </ruleml:Atom>

      </lrml:Obligation>

    </ruleml:then>

</ruleml:Rule>

</lrml:ConstitutiveStatement>"""

### Part-3-A

Trying an approach for prompting using SystemMessagePromptTemplate

In [16]:
def RAGPrompt(hf):
    
    sys_prompt = """You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. 
        The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.
        """
    '''
    human_prompt = """
    ### [INST] 
        <<SYS>>
        {system_prompt}
        
        The category being implemented is defined as follows:
        
        {metamodel_definition}
        
         <</SYS>>
        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definition of the metamodel provided in the LegalRuleML specification.
    
        {metamodel_XML}
        
    
        ### LEGAL_XML:
        {question} 
        [/INST]
    """
    '''
    
    
    
    human_prompt = """
    ### [INST] 
        
        The category being implemented is defined as follows:
        
        {metamodel_definition}
        
        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definition of the metamodel provided in the LegalRuleML specification.
    
        {metamodel_XML}
        
    
        ### LEGAL_XML:
        {question} 
        [/INST]
        """
    
    
    def get_relevant_metamodel(retrieverObj):
        docs = retrieverObj.get_relevant_documents("query")

        docs = docs.replace('\n', '')
        docs = docs.replace('\t', '')

        data = json.loads(docs)

        relevant_metamodel = data[2]["deontic"]
        return relevant_metamodel

    def get_relevant_definition(retrieverObj):
        docs = retrieverObj.get_relevant_documents("query")
        docs = docs.replace('\n', '')
        docs = docs.replace('\t', '')

        data = json.loads(docs)

        relevant_definition = data[2]["description"]
        return relevant_definition
    
   
    
    # Assuming sys_prompt and human_prompt are defined earlier in your code
    template = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        ("human", human_prompt)
    ])
    
    #prompt=template.format_prompt()
    
    # Correctly initialize LLMChain with your language model handler ('hf') and the created template
    llm_chain = LLMChain(llm=hf, prompt=template)
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-Chat-GPTQ")

    # Assuming sys_prompt and human_prompt are strings containing your prompts
    messages = [sys_prompt, human_prompt]

    # Tokenize each message and sum their lengths
    total_tokens = sum([len(tokenizer.encode(message, add_special_tokens=True)) for message in messages])

    print("\n\n The prompt for input is ", template, "\n\n", "Length is", total_tokens, "\n\n")

    
    input_dict = {
                  "metamodel_definition": metamodel_definition,
                  "metamodel_XML": metamodel_XML,
                  "question": text_value,
                  }
    
    

    # Invoke the LLMChain with the dictionary as input
    new_text_value = llm_chain.invoke(input=input_dict)

    return new_text_value

In [9]:
metamodel_definition = "Defeasibility is the property that a conclusion is open in principle to revision in case more evidence to the contrary is provided. Defeasible reasoning is in contrast to monotonic reasoning of propositional logic, where no revision is possible. In addition, defeasible reasoning allows reasoning in the face of contradictions, which gives rise to ex false quodlibet in propositional logic. One application of defeasible reasoning is the ability to model exceptions in a simple and natural way."
metamodel_XML = """       
<?xml version="1.0" encoding="UTF-8"?>
 <!DOCTYPE RDF [
   <!ENTITY dcterms "http://purl.org/dc/terms">  
   <!ENTITY rdf "http://www.w3.org/1999/02/22-rdf-syntax-ns">  
   <!ENTITY rdfs "http://www.w3.org/2000/01/rdf-schema">  
   <!ENTITY owl "http://www.w3.org/2002/07/owl">    
   <!ENTITY xs "http://www.w3.org/2001/XMLSchema">    
   <!ENTITY rulemm "http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel">  
   <!ENTITY lrmlmm "http://docs.oasis-open.org/legalruleml/ns/v1.0/metamodel">    
   <!ENTITY lrmlv "http://docs.oasis-open.org/legalruleml/ns/v1.0/vocab">  
 ]>
  
  <rdfs:Class rdf:about="#Override">
    <rdfs:label>override</rdfs:label>
    <rdfs:comment>an indication that a Legal Rule takes precedence over another Legal Rule. The ordered pair of Legal Rules is an instance in a defeasible priority relation.</rdfs:comment>
    <rdfs:subClassOf rdf:resource="&rulemm;#Atom"/>
  </rdfs:Class>

  <!-- Import from RuleML for diagramming purposes -->  
  <rdfs:Class rdf:about="&rulemm;#Atom">
    <rdfs:subClassOf rdf:resource="&rulemm;#Formula"/>
  </rdfs:Class>
  
  <rdfs:Class rdf:about="#Strength">
    <rdfs:label>strength</rdfs:label>
    <rdfs:comment>the quality of a Legal Rule to resist or not to resist a rebuttal.</rdfs:comment>
    <rdfs:subClassOf rdf:resource="#Associable"/>
  </rdfs:Class>
  

</rdf:RDF>
"""

text_value = result

In [17]:
new_text_value = RAGPrompt(hf)



 The prompt for input is  input_variables=['metamodel_XML', 'metamodel_definition', 'question'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. \n        The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.\n        ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['metamodel_XML', 'metamodel_definition', 'question'], template='\n    ### [INST] \n        \n        The category being implemented is defined as follows:\n        \n        {metamodel_definition}\n        \n        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definition of the metamodel provided in the LegalRuleML specification.\n    \n        {metamodel_XML}\n 

In [18]:
print(new_text_value)

{'metamodel_definition': 'Defeasibility is the property that a conclusion is open in principle to revision in case more evidence to the contrary is provided. Defeasible reasoning is in contrast to monotonic reasoning of propositional logic, where no revision is possible. In addition, defeasible reasoning allows reasoning in the face of contradictions, which gives rise to ex false quodlibet in propositional logic. One application of defeasible reasoning is the ability to model exceptions in a simple and natural way.', 'metamodel_XML': '       \n<?xml version="1.0" encoding="UTF-8"?>\n <!DOCTYPE RDF [\n   <!ENTITY dcterms "http://purl.org/dc/terms">  \n   <!ENTITY rdf "http://www.w3.org/1999/02/22-rdf-syntax-ns">  \n   <!ENTITY rdfs "http://www.w3.org/2000/01/rdf-schema">  \n   <!ENTITY owl "http://www.w3.org/2002/07/owl">    \n   <!ENTITY xs "http://www.w3.org/2001/XMLSchema">    \n   <!ENTITY rulemm "http://docs.oasis-open.org/legalruleml/ns/v1.0/rule-metamodel">  \n   <!ENTITY lrmlmm 

### Part-3-B
Prompting the ChatPromptTemplate by adding System Prompt to Llama2's instruction prompt

In [19]:
def RAGPrompt(hf):
    
    sys_prompt = """You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. 
        The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.
        """
    
    
    #Since the context window did not have system_prompt, it was returning a deficient output (asking for more information or context)
    # In this strategy, the human_prompt itself is being used to provide the system prompt as context, using the tags accepted by Llama2 (see https://huggingface.co/blog/llama2)
    
    human_prompt = """
    ### [INST] 
        <<SYS>>
        {system_prompt}
        
        The category being implemented is defined as follows:
        
        {metamodel_definition}
        
         <</SYS>>
        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definition of the metamodel provided in the LegalRuleML specification.
    
        {metamodel_XML}
        
    
        ### LEGAL_XML:
        {question} 
        [/INST]
    """
    
    
    '''
    
    human_prompt = """
    ### [INST] 
        
        The category being implemented is defined as follows:
        
        {metamodel_definition}
        
        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definition of the metamodel provided in the LegalRuleML specification.
    
        {metamodel_XML}
        
    
        ### LEGAL_XML:
        {question} 
        [/INST]
        """
    '''
    
    def get_relevant_metamodel(retrieverObj):
        docs = retrieverObj.get_relevant_documents("query")

        docs = docs.replace('\n', '')
        docs = docs.replace('\t', '')

        data = json.loads(docs)

        relevant_metamodel = data[2]["deontic"]
        return relevant_metamodel

    def get_relevant_definition(retrieverObj):
        docs = retrieverObj.get_relevant_documents("query")
        docs = docs.replace('\n', '')
        docs = docs.replace('\t', '')

        data = json.loads(docs)

        relevant_definition = data[2]["description"]
        return relevant_definition
    
   
    
    # Assuming sys_prompt and human_prompt are defined earlier in your code
    template = ChatPromptTemplate.from_messages([
        ("system", sys_prompt),
        ("human", human_prompt)
    ])
    
    #prompt=template.format_prompt()
    
    # Correctly initialize LLMChain with your language model handler ('hf') and the created template
    llm_chain = LLMChain(llm=hf, prompt=template)
    
    tokenizer = AutoTokenizer.from_pretrained("TheBloke/Llama-2-7B-Chat-GPTQ")

    # Assuming sys_prompt and human_prompt are strings containing your prompts
    messages = [sys_prompt, human_prompt]

    # Tokenize each message and sum their lengths
    total_tokens = sum([len(tokenizer.encode(message, add_special_tokens=True)) for message in messages])

    print("\n\n The prompt for input is ", template, "\n\n", "Length is", total_tokens, "\n\n")


    
    input_dict = {
                  "system_prompt": sys_prompt,
                  "metamodel_definition": metamodel_definition,
                  "metamodel_XML": metamodel_XML,
                  "question": text_value,
                  }

    # Invoke the LLMChain with the dictionary as input
    new_text_value = llm_chain.invoke(input=input_dict)

    return new_text_value

In [20]:
new_text_value = RAGPrompt(hf)



 The prompt for input is  input_variables=['metamodel_XML', 'metamodel_definition', 'question', 'system_prompt'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. \n        The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.\n        ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['metamodel_XML', 'metamodel_definition', 'question', 'system_prompt'], template='\n    ### [INST] \n        <<SYS>>\n        {system_prompt}\n        \n        The category being implemented is defined as follows:\n        \n        {metamodel_definition}\n        \n         <</SYS>>\n        Use the following metamodel in RDFS to ensure that the XML generated is in terms of the definitio

In [21]:
print(new_text_value)

{'system_prompt': 'You are an assistant that takes the XML generated and tweaks it in accordance with the rdfs metamodel provided to you below as context. \n        The XML is in compliance with the LegalRuleML core specification, whereas the metamodel provides an overview of what the XML should be for a particular category of XML elements.\n        ', 'metamodel_definition': 'Defeasibility is the property that a conclusion is open in principle to revision in case more evidence to the contrary is provided. Defeasible reasoning is in contrast to monotonic reasoning of propositional logic, where no revision is possible. In addition, defeasible reasoning allows reasoning in the face of contradictions, which gives rise to ex false quodlibet in propositional logic. One application of defeasible reasoning is the ability to model exceptions in a simple and natural way.', 'metamodel_XML': '       \n<?xml version="1.0" encoding="UTF-8"?>\n <!DOCTYPE RDF [\n   <!ENTITY dcterms "http://purl.org/d